In [1]:
import sklearn
from sklearn import datasets
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn import metrics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

C:\Users\patri\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# Read weather and load csv
# Drop "HE" and nulls
# Add a day and hour count variable
dataset = pd.read_csv("weather and load.csv")
dataset = pd.DataFrame(dataset)
dataset = dataset.drop(columns="HE")
dataset = dataset.dropna()

for x in range(8):
    dataset[f"day_{x}"] = (dataset["Day Of Week"] == x) *1
dataset["hour_count"] = list(range(len(dataset)))
dataset["hour"] = dataset.Dates.apply(lambda x: int(x.split()[-1].split(":")[0]))
ML_csv = dataset.to_csv("ML variables.csv", index=True)
dataset.head()

,Dates,Day Of Week,Temp,Dew Point,Humidity,Cloud Cover,Load,day_0,day_1,day_2,day_3,day_4,day_5,day_6,day_7,hour_count,hour
0,1/1/2014 0:00,4,41,35,79,30,2096,0,0,0,0,1,0,0,0,0,0
1,1/1/2014 1:00,4,41,34,76,30,1986,0,0,0,0,1,0,0,0,1,1
2,1/1/2014 2:00,4,39,31,73,30,1936,0,0,0,0,1,0,0,0,2,2
3,1/1/2014 3:00,4,39,32,76,30,1896,0,0,0,0,1,0,0,0,3,3
4,1/1/2014 4:00,4,41,32,70,30,1899,0,0,0,0,1,0,0,0,4,4


In [3]:
# Split date and hour into separate columns
dataset.Dates[0].split()

['1/1/2014', '0:00']

In [5]:
# Set X and y 
X = dataset[
    ["Temp","Cloud Cover", "Humidity", "Dew Point",
     "day_0", "day_1", "day_2", "day_3", "day_4", "day_5", 
     "day_6", "day_7", "hour_count", "hour"]]
y = dataset[["Load"]]

# Create TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=4)
print(tscv)  

# Loop through TimeSeriesSplits and train RandomForrestRegressor
for train_index, test_index in tscv.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Set RFR params
    rfr = RandomForestRegressor(max_features=.75, max_depth=8, min_samples_leaf=3, max_leaf_nodes=200)
    rfr.fit(X_train, y_train)
    print(rfr.score(X_train, y_train))
    print(rfr.score(X_test, y_test))

TimeSeriesSplit(max_train_size=None, n_splits=4)
TRAIN: [   0    1    2 ... 8764 8765 8766] TEST: [ 8767  8768  8769 ... 17527 17528 17529]
0.9607513119567581
0.7943388040795165
TRAIN: [    0     1     2 ... 17527 17528 17529] TEST: [17530 17531 17532 ... 26290 26291 26292]


C:\Users\patri\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
C:\Users\patri\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


0.9446133839377079
0.7913448222641697
TRAIN: [    0     1     2 ... 26290 26291 26292] TEST: [26293 26294 26295 ... 35053 35054 35055]


C:\Users\patri\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


0.9284267470310407
0.7107864092291463
TRAIN: [    0     1     2 ... 35053 35054 35055] TEST: [35056 35057 35058 ... 43816 43817 43818]


C:\Users\patri\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


0.9153370969522089
0.822302102484802


In [166]:
#Create DF of the actual and predicted loads
pred_y = rfr.predict(X_test)

df = pd.DataFrame({ "Hour Count" : X_test.hour_count,
                    "Actual: " : y_test.values.ravel(),
                    "Predicted: " : pred_y.ravel()})
df.head()

,Hour Count,Actual:,Predicted:
35056,35056,1911,1883.807674
35057,35057,1963,1887.588304
35058,35058,2033,1952.670918
35059,35059,2132,2101.429023
35060,35060,2304,2514.380864


In [167]:
# Print final R^2 value
print(rfr.score(X_test, y_test))

0.8267770038178446


In [168]:
# Save CSV
export_csv = df.to_csv("Actual vs Predicted Load.csv", index=True)

In [169]:
# Write/Pickle Model
rf_model = open("Random_Forest_Model.pkl", "wb")
pickle.dump(rfr, rf_model)
rf_model.close()

In [170]:
# Read Model from Pickle
import_model = open("Random_Forest_Model.pkl", "rb")
model = pickle.load(import_model)
model

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=8,
           max_features=0.75, max_leaf_nodes=200,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=3, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [171]:
# Check Features
model.feature_importances_

array([2.17316676e-01, 3.56445910e-03, 4.61272398e-02, 1.65585179e-01,
       0.00000000e+00, 2.51342610e-02, 1.23524776e-04, 7.21479195e-05,
       2.12454052e-05, 8.69977825e-05, 0.00000000e+00, 1.71256911e-02,
       6.13015689e-02, 4.63541009e-01])

In [173]:
# Temp, Dew Point, Humidity, Cloud Cover, 
# day_0, day_1, day_2, day_3, day_4, day_5,
# day_6, day_7, day_count, hour
# Actual Load for below conditions = 1899
weather_conditions = [[41, 32, 70, 30, 0, 0, 0, 0, 1, 0, 0, 0, 4, 4]]
model.predict(weather_conditions)

array([1852.86881169])